In [9]:
from pymongo import MongoClient,TEXT
import matplotlib.pyplot as plt
import urllib.parse
import pandas as pd
import urllib.robotparser
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/linuxperso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
client=MongoClient('localhost', port=27017)

## Requête avec uniquement la Regex

In [11]:
import time


t1=time.time()
list_match="trump"#"france"#"climat"#"cop.?28"
database="scrapping"
collection_sitemaps="sitemaps"
collection_htmls="htmls"

def match_document_v1(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {"$match" : 
                                                                {
                                                                    "$and":
                                                                        [
                                                                            {
                                                                                "url" : # doit contenir la liste de mots clés
                                                                                        { 
                                                                                            '$regex' : '^(.*({list_match})).*$'.format(list_match=list_match),
                                                                                            '$options' : 'i'
                                                                                        },  
                                                                            }
                                                                        ]
                                                                }
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1,
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v1=match_document_v1(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)


0.5039534568786621


In [12]:
df_urls_match_v1

,url,mots_in_url,id_media,media_name
0,https://benzinga.com/sec/insider-trades/nltx/p...,"[sec, insider, trades, nltx, peter, m, strump]",2599,Benzinga
1,https://www.businessinsider.com/new-york-depos...,"[new, york, deposition, videos, show, annoyed,...",2602,Business Insider
2,https://www.businessinsider.com/trump-nikki-ha...,"[rump, nikki, haley, south, carolina, gop, pri...",2602,Business Insider
3,https://www.businessinsider.com/elise-stefanik...,"[elise, stefanik, nikki, haley, vp, trump, gop...",2602,Business Insider
4,https://www.businessinsider.com/bidens-campaig...,"[bidens, campaign, taunt, trump, confuses, nik...",2602,Business Insider
...,...,...,...,...
451,https://capitolhillcorner.org/2022/07/07/july-...,"[2022, 07, 07, july, 8, friday, night, jazz, a...",2929,Capitol Hill Corner
452,https://capitolhillcorner.org/2022/04/28/frida...,"[2022, 04, 28, friday, night, live, jazz, at, ...",2929,Capitol Hill Corner
453,https://www.msnbc.com/donald-trump,"[donald, trump]",2587,MSNBC
454,https://www.msnbc.com/trump-special-counsel-in...,"[rump, special, counsel, investigations, jan, ...",2587,MSNBC


# Requete avec uniquement le match sur les elemtn de l'array parsé

In [13]:
t1=time.time()
def match_document_v2(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                {
                                                                    "mots_in_url":list_match
                                                                },
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v2=match_document_v2(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)

0.09527969360351562


In [14]:
df_urls_match_v2

,url,mots_in_url,id_media,media_name
0,https://www.businessinsider.com/new-york-depos...,"[new, york, deposition, videos, show, annoyed,...",2602,Business Insider
1,https://www.businessinsider.com/elise-stefanik...,"[elise, stefanik, nikki, haley, vp, trump, gop...",2602,Business Insider
2,https://www.businessinsider.com/bidens-campaig...,"[bidens, campaign, taunt, trump, confuses, nik...",2602,Business Insider
3,https://www.businessinsider.com/wes-moore-mary...,"[wes, moore, maryland, first, year, service, i...",2602,Business Insider
4,https://www.businessinsider.com/3-bedroom-suit...,"[3, bedroom, suite, ireland, trump, hotel, wor...",2602,Business Insider
...,...,...,...,...
372,https://www.petworthnews.org/blog/petworth-tru...,"[blog, petworth, trump, snarky]",2940,Petworth News
373,https://www.petworthnews.org/blog/21-trump-street,"[blog, 21, trump, stree]",2940,Petworth News
374,https://www.petworthnews.org/blog/trump-alcoho...,"[blog, trump, alcohol, license, complain]",2940,Petworth News
375,https://www.petworthnews.org/blog/abra-decides...,"[blog, abra, decides, trump, hote]",2940,Petworth News


In [15]:
fusion=df_urls_match_v1.merge(df_urls_match_v2, how="outer", on=['url','url'])
fusion.loc[fusion.isna().any(axis=1)]

,url,mots_in_url_x,id_media_x,media_name_x,mots_in_url_y,id_media_y,media_name_y
0,https://benzinga.com/sec/insider-trades/nltx/p...,"[sec, insider, trades, nltx, peter, m, strump]",2599,Benzinga,NaN,NaN,NaN
2,https://www.businessinsider.com/trump-nikki-ha...,"[rump, nikki, haley, south, carolina, gop, pri...",2602,Business Insider,NaN,NaN,NaN
8,https://www.businessinsider.com/melania-trumps...,"[elania, trumps, best, friend, suzanne, ircha,...",2602,Business Insider,NaN,NaN,NaN
9,https://www.businessinsider.com/trump-wants-mi...,"[rump, wants, mistrial, e, jean, carroll, defa...",2602,Business Insider,NaN,NaN,NaN
11,https://www.businessinsider.com/trump-nikki-ha...,"[rump, nikki, haley, new, hampshire, open, pri...",2602,Business Insider,NaN,NaN,NaN
...,...,...,...,...,...,...,...
452,https://www.petworthnews.org/blog/tag/Trump+Hotel,"[blog, tag, trump+hote]",2940,Petworth News,NaN,NaN,NaN
453,https://capitolhillcorner.org/2022/07/07/july-...,"[2022, 07, 07, july, 8, friday, night, jazz, a...",2929,Capitol Hill Corner,NaN,NaN,NaN
454,https://capitolhillcorner.org/2022/04/28/frida...,"[2022, 04, 28, friday, night, live, jazz, at, ...",2929,Capitol Hill Corner,NaN,NaN,NaN
456,https://www.msnbc.com/trump-special-counsel-in...,"[rump, special, counsel, investigations, jan, ...",2587,MSNBC,NaN,NaN,NaN


# Requête avec création d'un index de text

In [25]:
if "url_text" in client[database][collection_htmls].index_information():
    client[database][collection_htmls].drop_index('url_text')

client[database][collection_htmls].create_index([('mots_in_url', TEXT)], default_language='english')

'mots_in_url_text'

In [ ]:
t1=time.time()
def match_document_v3(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                { "$text": { "$search": list_match } } 
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v3=match_document_v3(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)

0.008987665176391602


In [ ]:
df_urls_match_v3


,url,mots_in_url,id_media,media_name
0,https://truthout.org/articles/former-trump-law...,"[articles, former, trump, lawyers, revelation,...",2747,Truth Out
1,https://www.inquisitr.com/heres-why-donald-tru...,"[eres, why, donald, trump, cant, stop, mention...",2682,Inquisitr
2,https://thesource.com/2020/11/09/former-trump-...,"[2020, 11, 09, former, trump, aide, says, mela...",2618,The Source
3,https://www.businessinsider.com/ted-cruz-endor...,"[ed, cruz, endorsed, trump, after, trump, said...",2602,Business Insider
4,https://talkingpointsmemo.com/where-things-sta...,"[where, things, stand, where, things, stand, t...",2745,TPM
...,...,...,...,...
441,https://www.theguardian.com/lifeandstyle/audio...,"[ifeandstyle, audio, 2024, jan, 20, weekend, p...",2847,Guardian
442,https://i0.wp.com/www.nationalreview.com/wp-co...,"[www.nationalreview.com, wp, content, uploads,...",2812,National Review
443,https://thesource.com/info@example.com?&subjec...,[info@example.co],2618,The Source
444,https://www.telemundowashingtondc.com/noticias...,"[noticias, decision, nikki, haley, se, pregunt...",2906,Telemundo Washington DC


In [ ]:
fusion=df_urls_match_v1.merge(df_urls_match_v3, how="outer", on=['url','url'])
fusion.loc[fusion.isna().any(axis=1)]

,url,mots_in_url_x,id_media_x,media_name_x,mots_in_url_y,id_media_y,media_name_y
0,https://benzinga.com/sec/insider-trades/nltx/p...,"[sec, insider, trades, nltx, peter, m, strump]",2599,Benzinga,NaN,NaN,NaN
75,https://static01.nyt.com/images/2024/01/19/mul...,"[images, 2024, 01, 19, multimedia, 19newsquiz,...",2848,New York Times,NaN,NaN,NaN
124,https://talkingpointsmemo.com/edblog/always-at...,"[edblog, always, attack, some, thoughts, on, p...",2745,TPM,NaN,NaN,NaN
205,https://talkingpointsmemo.com/edblog/the-saudi...,"[edblog, the, saudi, russian, trumpy, alliance]",2745,TPM,NaN,NaN,NaN
245,https://talkingpointsmemo.com/newsletter/vol-2...,"[newsletter, vol, 2, no, 60, are, trumpite, le...",2745,TPM,NaN,NaN,NaN
316,https://truthout.org/articles/fani-willis-accu...,"[articles, fani, willis, accuses, trumpworld, ...",2747,Truth Out,NaN,NaN,NaN
334,https://truthout.org/articles/the-heritage-fou...,"[articles, the, heritage, foundation, is, prep...",2747,Truth Out,NaN,NaN,NaN
449,https://images.squarespace-cdn.com/content/v1/...,"[content, v1, 5522ceeae4b0593541222134, 152259...",2940,Petworth News,NaN,NaN,NaN
453,https://capitolhillcorner.org/2022/07/07/july-...,"[2022, 07, 07, july, 8, friday, night, jazz, a...",2929,Capitol Hill Corner,NaN,NaN,NaN
454,https://capitolhillcorner.org/2022/04/28/frida...,"[2022, 04, 28, friday, night, live, jazz, at, ...",2929,Capitol Hill Corner,NaN,NaN,NaN


# Requête avec index de texte et regex

In [ ]:
t1=time.time()
def match_document_v4(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                {
                                                                    "$and":
                                                                            [
                                                                                { "$text": { "$search": list_match } },
                                                                                {
                                                                                "url" : # doit contenir la liste de mots clés
                                                                                        { 
                                                                                            '$regex' : '^(.*({list_match})).*$'.format(list_match=list_match),
                                                                                            '$options' : 'i'
                                                                                        },  
                                                                                }
                                                                            ]
                                                                },
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v4=match_document_v4(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)

0.023534059524536133


In [ ]:
df_urls_match_v4

,url,mots_in_url,id_media,media_name
0,https://truthout.org/articles/former-trump-law...,"[articles, former, trump, lawyers, revelation,...",2747,Truth Out
1,https://www.inquisitr.com/heres-why-donald-tru...,"[eres, why, donald, trump, cant, stop, mention...",2682,Inquisitr
2,https://thesource.com/2020/11/09/former-trump-...,"[2020, 11, 09, former, trump, aide, says, mela...",2618,The Source
3,https://www.businessinsider.com/ted-cruz-endor...,"[ed, cruz, endorsed, trump, after, trump, said...",2602,Business Insider
4,https://talkingpointsmemo.com/where-things-sta...,"[where, things, stand, where, things, stand, t...",2745,TPM
...,...,...,...,...
441,https://www.theguardian.com/lifeandstyle/audio...,"[ifeandstyle, audio, 2024, jan, 20, weekend, p...",2847,Guardian
442,https://i0.wp.com/www.nationalreview.com/wp-co...,"[www.nationalreview.com, wp, content, uploads,...",2812,National Review
443,https://thesource.com/info@example.com?&subjec...,[info@example.co],2618,The Source
444,https://www.telemundowashingtondc.com/noticias...,"[noticias, decision, nikki, haley, se, pregunt...",2906,Telemundo Washington DC


In [ ]:
client[database][collection_htmls].index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'mots_in_url_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('mots_in_url', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [ ]:
client[database][collection_htmls].drop_index('mots_in_url_text')

In [ ]:
client[database][collection_htmls].create_index([('url', TEXT)], default_language='english')

'url_text'

In [ ]:
t1=time.time()
def match_document_v5(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                { "$text": { "$search": list_match } } 
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v5=match_document_v3(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)

0.020941495895385742


In [ ]:
df_urls_match_v5

,url,mots_in_url,id_media,media_name
0,https://truthout.org/articles/former-trump-law...,"[articles, former, trump, lawyers, revelation,...",2747,Truth Out
1,https://www.inquisitr.com/heres-why-donald-tru...,"[eres, why, donald, trump, cant, stop, mention...",2682,Inquisitr
2,https://thesource.com/2020/11/09/former-trump-...,"[2020, 11, 09, former, trump, aide, says, mela...",2618,The Source
3,https://www.businessinsider.com/ted-cruz-endor...,"[ed, cruz, endorsed, trump, after, trump, said...",2602,Business Insider
4,https://talkingpointsmemo.com/where-things-sta...,"[where, things, stand, where, things, stand, t...",2745,TPM
...,...,...,...,...
441,https://www.theguardian.com/lifeandstyle/audio...,"[ifeandstyle, audio, 2024, jan, 20, weekend, p...",2847,Guardian
442,https://i0.wp.com/www.nationalreview.com/wp-co...,"[www.nationalreview.com, wp, content, uploads,...",2812,National Review
443,https://thesource.com/info@example.com?&subjec...,[info@example.co],2618,The Source
444,https://www.telemundowashingtondc.com/noticias...,"[noticias, decision, nikki, haley, se, pregunt...",2906,Telemundo Washington DC


In [ ]:
fusion=df_urls_match_v1.merge(df_urls_match_v5, how="outer", on=['url','url'])
fusion.loc[fusion.isna().any(axis=1)]

,url,mots_in_url_x,id_media_x,media_name_x,mots_in_url_y,id_media_y,media_name_y
0,https://benzinga.com/sec/insider-trades/nltx/p...,"[sec, insider, trades, nltx, peter, m, strump]",2599,Benzinga,NaN,NaN,NaN
75,https://static01.nyt.com/images/2024/01/19/mul...,"[images, 2024, 01, 19, multimedia, 19newsquiz,...",2848,New York Times,NaN,NaN,NaN
124,https://talkingpointsmemo.com/edblog/always-at...,"[edblog, always, attack, some, thoughts, on, p...",2745,TPM,NaN,NaN,NaN
205,https://talkingpointsmemo.com/edblog/the-saudi...,"[edblog, the, saudi, russian, trumpy, alliance]",2745,TPM,NaN,NaN,NaN
245,https://talkingpointsmemo.com/newsletter/vol-2...,"[newsletter, vol, 2, no, 60, are, trumpite, le...",2745,TPM,NaN,NaN,NaN
316,https://truthout.org/articles/fani-willis-accu...,"[articles, fani, willis, accuses, trumpworld, ...",2747,Truth Out,NaN,NaN,NaN
334,https://truthout.org/articles/the-heritage-fou...,"[articles, the, heritage, foundation, is, prep...",2747,Truth Out,NaN,NaN,NaN
449,https://images.squarespace-cdn.com/content/v1/...,"[content, v1, 5522ceeae4b0593541222134, 152259...",2940,Petworth News,NaN,NaN,NaN
453,https://capitolhillcorner.org/2022/07/07/july-...,"[2022, 07, 07, july, 8, friday, night, jazz, a...",2929,Capitol Hill Corner,NaN,NaN,NaN
454,https://capitolhillcorner.org/2022/04/28/frida...,"[2022, 04, 28, friday, night, live, jazz, at, ...",2929,Capitol Hill Corner,NaN,NaN,NaN
